# Visualising the traffic around Boston Airport

Aim of this notebook is to provide a codealong to be able to recreate the traffic from Sheepfold Dog Park in Stoneham, Massachusetts to Boston Logan Airport.

In [1]:
# Importing the necessary packages:
import pandas as pd
import numpy as np
import json
import requests

In [74]:
with open('Parkresponse.json', 'r') as f:
    directions = json.load(f)

In [137]:
def retrieve_weighted_root(json):
    """
    Input: json file retrieved from googlemaps directions API after a request has
    been sent to receive directions for a route between point A and point B. 
    ---------------------------------------------------------
    Outputs: weighted coordinates for the different steps along your route in a 
    dataframe format and the ETA at your destination.
    """

    route_dict = json['routes']
    
    duration_in_traffic = None
    root_steps = []
    for legs in route:
        for steps in legs['legs']:
            for coordinates in steps['steps']:
                root_steps.append(coordinates['start_location'])
            root_steps.append(coordinates['end_location'])
        duration_in_traffic = steps['duration_in_traffic']['text']

#Chosen to only select the start coordinate of each step given that the end coordinate of 
#step 0 is the start coordinate of step 1. Last coordinate is added at the end of the loop.

    weighted_coordinates = ([(root_step['lat'], 
                              root_step['lng'],
                              np.random.randint(1,4)) for root_step in root_steps])
    
    weighted_coordinates_df = pd.DataFrame(weighted_coordinates,
                                           index=range(1, len(weighted_coordinates)+1),
                                           columns=['latitude', 'longitude', 'weight'])
    
    return weighted_coordinates_df, duration_in_traffic

In [138]:
weighted_coordinates, duration_in_traffic = retrieve_weighted_root(directions)

In [139]:
weighted_coordinates

,latitude,longitude,weight
1,42.453338,-71.106772,3
2,42.451445,-71.104786,1
3,42.451971,-71.104637,2
4,42.363433,-71.057959,3
5,42.362142,-71.056596,3
6,42.362611,-71.053952,2
7,42.372470,-71.034031,1
8,42.371191,-71.027445,1
9,42.364970,-71.020226,3
10,42.365821,-71.018036,2


In [10]:
def get_keys(path):
    """
    Input: path from my local computer to the files where api keys are stored.
    -----------------------------------------------------------
    Output: access to my the desired file.
    """
    
    with open(path) as f:
        return json.load(f)
    
keys = get_keys("/Users/jjherranzsarrion/.secret/google_blog2_api.json")

In [125]:
import gmaps
import gmaps.datasets
%matplotlib inline

gmaps.configure(api_key=keys['api_key'])

def colour_coding_line(weighted_coordinates, duration_in_traffic):
    """
    Inputs: a dataframe that contains coordinates for a route between point A and point B,
    and the ETA at your destination.
    -----------------------------------------------------------
    Output: a polyline between point A and point B overlaying google maps as well as 
    the traffic information between these points represented by such polyline colour coded.
    """
    
    fig = gmaps.figure()

    origin_destination = [{'name': ('Origin', 'The moment I realised the state of traffic'), 
                           'location': (weighted_coordinates['latitude'][1],
                                        weighted_coordinates['longitude'][1]),
                           'ETA': duration_in_traffic},
                          {'name': ('Destination', 'Boston Logan Airport'), 
                           'location': (weighted_coordinates['latitude'][len(weighted_coordinates_df)],
                                        weighted_coordinates['longitude'][len(weighted_coordinates_df)]),
                           'ETA': duration_in_traffic}]

    markers_location = [marker['location'] for marker in origin_destination]
    info_box_template = """
    <dt>{name[0]}</dt><dd>{name[1]}<dd> 
    <dt>ETA:</dt>{ETA}<dd></p>
    """
    markers_info = [info_box_template.format(**marker) for marker in origin_destination]
    marker_layer = gmaps.marker_layer(markers_location, info_box_content=markers_info)
    fig.add_layer(marker_layer)
    
    for i in range(1, len(weighted_coordinates)):
        origin = (weighted_coordinates['latitude'][i], weighted_coordinates['longitude'][i])
        origin_weight = weighted_coordinates['weight'][i]
        destination = (weighted_coordinates['latitude'][i+1], weighted_coordinates['longitude'][i+1])
        destination_weight = weighted_coordinates['weight'][i+1]
        
        if origin_weight == 3:
            colour = 'red'
        elif origin_weight == 2:
            colour = (255,165,0) #tuple to represent orange
        else:
            colour='green'
        
        fig.add_layer(gmaps.directions_layer(origin, 
                                                 destination, 
                                                 show_markers=False, 
                                                 stroke_color=colour, 
                                                 stroke_opacity=0.8))
    
    return fig

In [140]:
colour_coding_line(weighted_coordinates, duration_in_traffic)

Figure(layout=FigureLayout(height='420px'))